In [1]:
import datetime
import pandas as pd
from typing import Generator

The futures data have 2 kinds of order, normal tradings and calendar spreads ( or switches ).

The definition and difference won't be covered, please [`check`](https://en.wikipedia.org/wiki/Calendar_spread) .

However, the target of those dealt are identical. Therefore the price and volume will be taken together.

In [2]:
dealt_url_template = 'https://www.taifex.com.tw/file/taifex/Dailydownload/DailydownloadCSV/{}'
dealt_filename_template = 'Daily_{}.zip'

date = datetime.date(2022, 1, 20)
s3_bucket = 'indextracker'

target_contracts = ['TX', 'MTX']

dealt_filename = dealt_filename_template.format(date.strftime('%Y_%m_%d'))

dealt_s3_key = f'tw/raw/futures/{dealt_filename}'
dealt_url = dealt_url_template.format(dealt_filename)

dealt_url

'https://www.taifex.com.tw/file/taifex/Dailydownload/DailydownloadCSV/Daily_2022_01_20.zip'

To reduce overheads of unzipping files, unzip and read data from archives directly on the fly.

In [3]:
def get_zipped_objects_from_s3(bucket, key):
    """
    unzip a zip file and get content files as file-like objects on the fly from S3

    args:
        bucket - S3 bucket name
        key - S3 object key

    """
    import boto3, zipfile, io
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    with io.BytesIO(obj['Body'].read()) as f:
        with zipfile.ZipFile(f) as zf:
            for file in zf.namelist():
                yield zf.read(file)

In [4]:
dealt_content = list(get_zipped_objects_from_s3(s3_bucket, dealt_s3_key))[0]
dealt_arr = dealt_content.decode('big5').split('\r\n')

dealt_arr[0]

'成交日期,商品代號,到期月份(週別),成交時間,成交價格,成交數量(B+S),近月價格,遠月價格,開盤集合競價 '

In [5]:
dealt_arr_2d = [x.split(',') for x in dealt_arr[1:]]

dealt_df = pd.DataFrame(dealt_arr_2d)
dealt_df.columns = ['date', 'contract', 'expire', 'time', 'price', 'volume', 'near_price', 'far_price', 'is_open_auction']

dealt_df

,date,contract,expire,time,price,volume,near_price,far_price,is_open_auction
0,20220119,BRF,202203,182841,2441,10,-,-,
1,20220119,BRF,202203,183151,2442.5,10,-,-,
2,20220119,BRF,202203,184034,2444.5,10,-,-,
3,20220119,BRF,202203,190642,2440.5,30,-,-,
4,20220119,BRF,202203,191830,2440.5,2,-,-,
...,...,...,...,...,...,...,...,...,...
402106,20220120,ZFF,202203,112711,1785.6,2,-,-,
402107,20220120,ZFF,202203,124405,1787,2,-,-,
402108,20220120,ZFF,202203,124423,1787,2,-,-,
402109,20220120,ZFF,202203,132833,1784.6,2,-,-,


In [6]:
dealt_df['contract'] = dealt_df.contract.str.strip()
dealt_df = dealt_df[dealt_df['contract'].isin(target_contracts)]
dealt_df = dealt_df.drop('is_open_auction', axis=1)

dealt_df

,date,contract,expire,time,price,volume,near_price,far_price
53335,20220119,MTX,202201W4,150001,18184,4,-,-
53336,20220119,MTX,202201W4,150021,18178,2,-,-
53337,20220119,MTX,202201W4,150624,18188,2,-,-
53338,20220119,MTX,202201W4,150705,18188,2,-,-
53339,20220119,MTX,202201W4,151501,18189,2,-,-
...,...,...,...,...,...,...,...,...
390504,20220120,TX,202212,133712,17566,4,-,-
390505,20220120,TX,202212,133910,17574,2,-,-
390506,20220120,TX,202212,133910,17575,2,-,-
390507,20220120,TX,202212,134108,17571,2,-,-


Then, the calendar spreads

In [7]:
spread_url_template = 'https://www.taifex.com.tw/file/taifex/Dailydownload/DailydownloadCSV_C/{}'
spread_filename_template = 'Daily_{}_C.zip'

spread_filename = spread_filename_template.format(date.strftime('%Y_%m_%d'))

spread_s3_key = f'tw/raw/futures/{spread_filename}'
spread_url = spread_url_template.format(spread_filename)

spread_url

'https://www.taifex.com.tw/file/taifex/Dailydownload/DailydownloadCSV_C/Daily_2022_01_20_C.zip'

In [8]:
spread_content = list(get_zipped_objects_from_s3(s3_bucket, spread_s3_key))[0]
spread_arr = spread_content.decode('big5').split('\r\n')

spread_arr[0]

'成交日期,商品代號,到期月份(週別),成交時間,成交價格,成交數量(B+S),近月價格,遠月價格,屬價差對價差成交者 '

In [9]:
spread_arr_2d = [x.split(',') for x in spread_arr[1:]]

spread_df = pd.DataFrame(spread_arr_2d)
spread_df.columns = ['date', 'contract', 'expire', 'time', 'price', 'volume', 'near_price', 'far_price', 'is_with_spread']
spread_df

,date,contract,expire,time,price,volume,near_price,far_price,is_with_spread
0,20220120,CBF,202202/202203,084526,-.06,4,34.1,34.04,*
1,20220120,CBF,202202/202203,084527,-.06,4,34.1,34.04,*
2,20220120,CBF,202202/202203,084531,-.06,32,34.1,34.04,*
3,20220120,CBF,202202/202203,084534,-.04,4,34.1,34.06,*
4,20220120,CBF,202202/202203,084609,0,4,34.15,34.15,
...,...,...,...,...,...,...,...,...,...
10461,20220120,ZFF,202202/202203,112711,-1.8,4,1787.4,1785.6,
10462,20220120,ZFF,202202/202203,124405,-.8,4,1787.8,1787,
10463,20220120,ZFF,202202/202203,124423,-.8,4,1787.8,1787,
10464,20220120,ZFF,202202/202203,132833,-.6,4,1785.2,1784.6,


In [10]:
spread_df.contract = spread_df.contract.str.strip()
spread_df = spread_df.loc[spread_df.contract.isin(target_contracts)]
spread_df = spread_df.drop('is_with_spread', axis=1)

spread_df

,date,contract,expire,time,price,volume,near_price,far_price
4908,20220119,MTX,202201W4/202202,150001,-18,8,18184,18166
4909,20220119,MTX,202201W4/202202,150021,-17,4,18178,18161
4910,20220119,MTX,202201W4/202202,151712,-27,4,18199,18172
4911,20220119,MTX,202201W4/202202,152230,-29,4,18194,18165
4912,20220119,MTX,202201W4/202202,155824,-29,4,18180,18151
...,...,...,...,...,...,...,...,...
10404,20220120,TX,202209/202212,095700,-81,4,17588,17507
10405,20220120,TX,202209/202212,120148,-81,4,17643,17562
10406,20220120,TX,202209/202212,133335,-80,4,17633,17553
10407,20220120,TX,202209/202212,133443,-78,4,17636,17558


combine the both orders and transforming together

In [11]:
total_df = pd.concat([dealt_df, spread_df])

In [12]:
total_df.expire.unique()

array(['202201W4     ', '202202     ', '202202/202209', '202202/202203',
       '202203     ', '202204     ', '202206     ', '202209     ',
       '202212     ', '202202/202204', '202201W4/202202',
       '202201W4/202204', '202202/202206', '202202/202212',
       '202203/202204', '202203/202212', '202206/202209', '202203/202206',
       '202204/202206', '202209/202212'], dtype=object)

In [13]:
total_df['expire'] = total_df['expire'].str.strip()
total_df['volume'] = pd.to_numeric(total_df['volume'])
total_df['datetime'] = pd.to_datetime(total_df.date.str.strip() + total_df.time.str.strip(), format='%Y%m%d%H%M%S').dt.tz_localize('Asia/Taipei')

Unpack switch dealt

In [14]:
switch_df = total_df[total_df['expire'].str.contains('/')]

switch_df_near = switch_df.copy()
switch_df_near.expire = switch_df_near.expire.str.extract(r'([0-9W]+)/[0-9W]+')
switch_df_near.price = switch_df_near.near_price
switch_df_near.volume = switch_df_near.volume // 2

switch_df_far = switch_df.copy()
switch_df_far.expire = switch_df_far.expire.str.extract(r'[0-9W]+/([0-9W]+)')
switch_df_far.price = switch_df_far.far_price
switch_df_far.volume = switch_df_far.volume // 2

total_df = total_df.drop(switch_df.index)
total_df = pd.concat([total_df, switch_df_near, switch_df_far])

In [15]:
total_df['price'] = pd.to_numeric(total_df['price'])
total_df = total_df[['datetime', 'contract', 'expire', 'price', 'volume']]

total_df

,datetime,contract,expire,price,volume
53335,2022-01-19 15:00:01+08:00,MTX,202201W4,18184,4
53336,2022-01-19 15:00:21+08:00,MTX,202201W4,18178,2
53337,2022-01-19 15:06:24+08:00,MTX,202201W4,18188,2
53338,2022-01-19 15:07:05+08:00,MTX,202201W4,18188,2
53339,2022-01-19 15:15:01+08:00,MTX,202201W4,18189,2
...,...,...,...,...,...
10404,2022-01-20 09:57:00+08:00,TX,202212,17507,2
10405,2022-01-20 12:01:48+08:00,TX,202212,17562,2
10406,2022-01-20 13:33:35+08:00,TX,202212,17553,2
10407,2022-01-20 13:34:43+08:00,TX,202212,17558,2


The contracts with same expiration could be viewed as different by time.

Distinguish them by adding a code.

The contract traded are:
+ `TX` - spot month, 2 successive month, and 3 other quarter months.
+ `MTX` - weekly contracts, spot month, 2 successive month, and 3 other quarter months.

In [16]:
def get_monthly_settlement_date(date:datetime.date) -> datetime.date:
    """
    get the settlement date of monthly contract, with id the 3rd wedensday of the month.

    args:
        date - the issuing date
    return:
        the settlement date
    """
    return date.replace(day = 15 + (2 - date.weekday() + date.day - 1) % 7)

def get_next_weekly_settlement_date(date:datetime.date) -> datetime.date:
    """
    get the settlement date of next weekly contract, that is, next wedensday.
    A weekly contract are started at the begining of wedensday, and expired at the end of the next wedensday.

    """
    return date + datetime.timedelta(days= 1 + (1 - date.weekday()) % 7)

# for i in range(365):
#     print(get_next_settlement_date(datetime.date(2022, 1, 1) + datetime.timedelta(days=i)))

In [17]:
def _get_expiration_code_map(date:datetime.date) -> Generator[tuple[str, str], None, None]:
    """
    underlying generator function of get_expiration_code_map

    args:
        date - the issuing date
    return:
        key-value pairs of expire and expire code
    """

    def get_weekly_contract_name(date:datetime.date):
        """
        private function, get weekly contract name by date

        args:
            date - the issuing date
        return:
            name of weekly contract
        """
        week = 1 + (date.day - 1) // 7
        if week == 3: # the 3rd weekly contract IS the monthly contract
            return f'{date.year:4}{date.month:02}'
        else:
            return f'{date.year:4}{date.month:02}W{week:1}'

    # wedensday is the day old weekly contract settle, and the new weekly contract started.
    # on the day both weekly contract is traded.
    if date.weekday() == 2:
        yield ('W', get_weekly_contract_name(date))

    next_settlement = get_next_weekly_settlement_date(date)
    yield ('W', get_weekly_contract_name(next_settlement))

    # if the date of the month is later than 3rd wedensday, the monthly contract is the one settled next month.
    year = date.year
    month = date.month
    settlement_month = get_monthly_settlement_date(date)
    if date > settlement_month:
        month += 1

    expiration_codes = ['M', 'M+1', 'M+2', 'Q+1', 'Q+2', 'Q+3']

    for i in range(3):
        m = month - 1 + i # map to 0~11 for calculating
        yield (expiration_codes[i], f'{(year + (m // 12)):4}{((m % 12) + 1):02}')
    for i in range(3):
        m = month - 1 + (i + 1) * 3 + ( - month % 3)
        yield (expiration_codes[i + 3], f'{(year + (m // 12)):4}{((m % 12) + 1):02}')

# for i in range(365):
    # print(list(_get_expiration_code_map(datetime.date(2021, 1, 1) + datetime.timedelta(days=i))))

In [18]:
def get_expiration_code_map(date:datetime.date):
    """
    Get a dict that mapping the expires of contracts with the expire codes by the date.
    A contract could be viewed as weekly and monthly contract at the same time.

    args:
        date - the issuing date
    return:
        a dict as map.
    """
    maps = list(_get_expiration_code_map(date))
    unique = set([x[1] for x in maps])
    return {x : [y[0] for y in maps if y[1] == x] for x in unique}

get_expiration_code_map(datetime.date(2022, 2, 16))

{'202209': ['Q+2'],
 '202202W4': ['W'],
 '202202': ['W', 'M'],
 '202206': ['Q+1'],
 '202203': ['M+1'],
 '202212': ['Q+3'],
 '202204': ['M+2']}

In [19]:
total_df.expire.unique()

array(['202201W4', '202202', '202203', '202204', '202206', '202209',
       '202212'], dtype=object)

In [20]:
expiration_code_map = get_expiration_code_map(date)

total_df['expire_code'] = total_df.expire.map(expiration_code_map)
total_df = total_df.explode('expire_code')

total_df

,datetime,contract,expire,price,volume,expire_code
53335,2022-01-19 15:00:01+08:00,MTX,202201W4,18184,4,W
53336,2022-01-19 15:00:21+08:00,MTX,202201W4,18178,2,W
53337,2022-01-19 15:06:24+08:00,MTX,202201W4,18188,2,W
53338,2022-01-19 15:07:05+08:00,MTX,202201W4,18188,2,W
53339,2022-01-19 15:15:01+08:00,MTX,202201W4,18189,2,W
...,...,...,...,...,...,...
10404,2022-01-20 09:57:00+08:00,TX,202212,17507,2,Q+3
10405,2022-01-20 12:01:48+08:00,TX,202212,17562,2,Q+3
10406,2022-01-20 13:33:35+08:00,TX,202212,17553,2,Q+3
10407,2022-01-20 13:34:43+08:00,TX,202212,17558,2,Q+3


All fields are ready to be aggregated.

To draw a candle stick diagram, the next step is to aggregate the data by scales of time.

In [21]:
def roundup_to_minutes(dt:datetime.datetime, minutes:int = 1):
    """
    round up a datetime to a scale of minutes

    args:
        dt - issuing datetime
        minutes - the scale to rounded up
    return:
        rounded up datetime
    """
    return (dt + datetime.timedelta(seconds=-dt.timestamp() % (minutes*60))).astimezone(datetime.timezone.utc)

# roundup_to_minutes(datetime.datetime(2020,2,22,16,9,58), 10)

Grouping data by contract and expiration, then do aggregation.

In [22]:
gp = total_df.groupby(by=['contract', 'expire_code'])

single = gp.get_group(('TX', 'M')).copy()

single

,datetime,contract,expire,price,volume,expire_code
284834,2022-01-19 15:00:00+08:00,TX,202202,18158,266,M
284835,2022-01-19 15:00:00+08:00,TX,202202,18158,8,M
284836,2022-01-19 15:00:00+08:00,TX,202202,18157,2,M
284837,2022-01-19 15:00:00+08:00,TX,202202,18157,4,M
284838,2022-01-19 15:00:00+08:00,TX,202202,18159,2,M
...,...,...,...,...,...,...
10347,2022-01-20 13:37:12+08:00,TX,202202,18222,2,M
10348,2022-01-20 13:39:10+08:00,TX,202202,18230,2,M
10349,2022-01-20 13:39:10+08:00,TX,202202,18230,2,M
10350,2022-01-20 13:41:08+08:00,TX,202202,18227,2,M


In [23]:
single['scale'] = single.datetime.apply(roundup_to_minutes, minutes=30)

In [24]:
single.groupby('scale').agg({'price': ['first', 'max', 'min', 'last', 'mean', 'std'], 'volume': 'sum'})

price                                     \
                           first    max    min   last          mean   
scale                                                                 
2022-01-19 07:00:00+00:00  18158  18160  18149  18153  18154.235294   
2022-01-19 07:30:00+00:00  18151  18172  18135  18149  18158.165690   
2022-01-19 08:00:00+00:00  18159  18165  18147  18147  18155.507030   
2022-01-19 08:30:00+00:00  18157  18174  18136  18163  18159.186335   
2022-01-19 09:00:00+00:00  18166  18197  18165  18197  18182.728630   
2022-01-19 09:30:00+00:00  18194  18209  18186  18201  18197.483257   
2022-01-19 10:00:00+00:00  18192  18236  18192  18234  18215.570066   
2022-01-19 10:30:00+00:00  18232  18233  18209  18228  18220.988040   
2022-01-19 11:00:00+00:00  18214  18225  18210  18219  18215.577922   
2022-01-19 11:30:00+00:00  18225  18229  18212  18212  18221.271501   
2022-01-19 12:00:00+00:00  18212  18225  18211  18222  18218.990291   
2022-01-19 12:30:00+00:00  18221  18224  18213  18217  18218.513846   
2022-01-19 13:00:00+00:00  18216  18220  18201  18215  18209.363122   
2022-01-19 13:30:00+00:00  18215  18240  18215  18234  18232.432692   
2022-01-19 14:00:00+00:00  18235  18239  18224  18229  18230.856115   
2022-01-19 14:30:00+00:00  18238  18238  18222  18223  18228.916190   
2022-01-19 15:00:00+00:00  18226  18250  18216  18241  18234.261379   
2022-01-19 15:30:00+00:00  18240  18244  18211  18225  18227.646789   
2022-01-19 16:00:00+00:00  18212  18217  18187  18199  18201.024670   
2022-01-19 16:30:00+00:00  18206  18215  18179  18185  18193.545060   
2022-01-19 17:00:00+00:00  18200  18217  18190  18205  18202.919588   
2022-01-19 17:30:00+00:00  18215  18230  18214  18224  18222.753477   
2022-01-19 18:00:00+00:00  18229  18234  18211  18221  18220.148276   
2022-01-19 18:30:00+00:00  18221  18232  18214  18214  18223.986702   
2022-01-19 19:00:00+00:00  18218  18221  18206  18214  18213.481848   
2022-01-19 19:30:00+00:00  18212  18215  18174  18179  18188.277019   
2022-01-19 20:00:00+00:00  18174  18204  18171  18173  18186.391204   
2022-01-19 20:30:00+00:00  18191  18211  18181  18189  18195.053269   
2022-01-19 21:00:00+00:00  18184  18185  18152  18153  18166.244954   
2022-01-20 01:00:00+00:00  18203  18216  18153  18175  18182.289892   
2022-01-20 01:30:00+00:00  18176  18211  18081  18113  18144.736598   
2022-01-20 02:00:00+00:00  18152  18184  18138  18168  18158.251425   
2022-01-20 02:30:00+00:00  18160  18186  18130  18170  18163.036898   
2022-01-20 03:00:00+00:00  18162  18163  18112  18142  18136.381569   
2022-01-20 03:30:00+00:00  18147  18185  18131  18178  18160.210565   
2022-01-20 04:00:00+00:00  18177  18222  18171  18218  18200.119068   
2022-01-20 04:30:00+00:00  18216  18255  18208  18223  18232.777941   
2022-01-20 05:00:00+00:00  18229  18270  18224  18259  18247.070361   
2022-01-20 05:30:00+00:00  18251  18263  18233  18240  18244.802885   
2022-01-20 06:00:00+00:00  18233  18235  18208  18227  18219.518353   

                                     volume  
                                 std    sum  
scale                                        
2022-01-19 07:00:00+00:00   3.172257    458  
2022-01-19 07:30:00+00:00   8.787757   6580  
2022-01-19 08:00:00+00:00   4.573301   3666  
2022-01-19 08:30:00+00:00  10.796787   5008  
2022-01-19 09:00:00+00:00   8.297484   5928  
2022-01-19 09:30:00+00:00   5.348063   4686  
2022-01-19 10:00:00+00:00   9.825125   5206  
2022-01-19 10:30:00+00:00   5.969447   3548  
2022-01-19 11:00:00+00:00   3.092677   1902  
2022-01-19 11:30:00+00:00   5.143759   1888  
2022-01-19 12:00:00+00:00   3.488584   1224  
2022-01-19 12:30:00+00:00   2.907625   1032  
2022-01-19 13:00:00+00:00   5.139772   2646  
2022-01-19 13:30:00+00:00   5.122371   3808  
2022-01-19 14:00:00+00:00   3.789416   1996  
2022-01-19 14:30:00+00:00   3.926401   1436  
2022-01-19 15:00:00+00:00   7.798235   9864  
2022-01-19 15:30:00+00:00   8.712269   7308  
2022-01